In [1]:
import os, subprocess, json, textwrap, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
print('repo exists', repo.exists())
print(subprocess.run(['git','-C',str(repo),'status','--short'], capture_output=True, text=True).stdout)

repo exists True



In [2]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
log = subprocess.run(['git','-C',str(repo),'reflog','-n','30','--date=iso'], capture_output=True, text=True)
print(log.stdout)
print(log.stderr)

81ae1c6 HEAD@{2026-03-02 00:44:41 +0000}: commit: Mise à jour du notebook de récupération pour afficher l'état du dépôt et le reflog Git, avec ajout de la gestion des sorties.
336d5c4 HEAD@{2026-03-02 00:44:09 +0000}: commit: Mise à jour des métadonnées du notebook pour spécifier la version de Python et le noyau utilisé.
06496c5 HEAD@{2026-03-02 00:43:08 +0000}: commit: Ajout d'un notebook Python pour vérifier l'existence du dépôt et afficher l'état Git
81e1cf2 HEAD@{2026-03-02 00:33:29 +0000}: commit: Add quiz functionality with vehicle type and service selection
3044a6a HEAD@{2026-03-02 00:28:56 +0000}: commit: Ajout d'une version légère des vidéos mobiles pour améliorer la lecture sur les réseaux lents et mise à jour des scripts et du README en conséquence.
24eadef HEAD@{2026-03-02 00:27:42 +0000}: commit: Ajout d'un script pour générer une version mobile ultra-compatible des vidéos et mise à jour du README pour inclure des instructions sur son utilisation.
3ecb7b8 HEAD@{2026-03-02 

In [3]:
import subprocess, pathlib, os
repo = pathlib.Path('/workspaces/prime-detail')
base_commit = '3044a6a'

# Restore key files from commit before accidental overwrite
subprocess.run(['git','-C',str(repo),'checkout',base_commit,'--','index.html','mentions-legales.html','styles.css','README.md','tools/transcode-mobile-video.sh'], check=True)

# Remove accidentally introduced files if present
for f in ['style.css','script.js']:
    p = repo / f
    if p.exists():
        try:
            subprocess.run(['git','-C',str(repo),'rm','-f',f], check=False, capture_output=True, text=True)
        except Exception:
            pass
        if p.exists():
            p.unlink()

status = subprocess.run(['git','-C',str(repo),'status','--short'], capture_output=True, text=True)
print(status.stdout)

M  index.html
D  script.js
D  style.css



In [4]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
subprocess.run(['git','-C',str(repo),'checkout','--','tools/recover.ipynb'], check=False)
print(subprocess.run(['git','-C',str(repo),'status','--short'], capture_output=True, text=True).stdout)

M  index.html
D  script.js
D  style.css



In [5]:
import subprocess, pathlib, shutil
repo = pathlib.Path('/workspaces/prime-detail')
ffmpeg = shutil.which('ffmpeg')
print('ffmpeg:', ffmpeg)
if ffmpeg:
    cmd = ['bash', str(repo / 'tools' / 'transcode-mobile-video.sh')]
    proc = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
    print('exit:', proc.returncode)
    print(proc.stdout)
    print(proc.stderr)
else:
    print('FFMPEG_MISSING')
print('videos:', [p.name for p in sorted((repo / 'assets' / 'videos').glob('*'))])

ffmpeg: None
FFMPEG_MISSING
videos: ['Pub prime detail dacia Logan.MOV']


In [6]:
import shutil, subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
print('node:', shutil.which('node'))
print('npm:', shutil.which('npm'))
print('npx:', shutil.which('npx'))
for cmd in (['node','-v'], ['npm','-v'], ['npx','--version']):
    exe = shutil.which(cmd[0])
    if not exe:
        print(cmd[0], 'MISSING')
        continue
    p = subprocess.run(cmd, capture_output=True, text=True)
    print(cmd[0], '=>', (p.stdout or p.stderr).strip())

node: /home/codespace/nvm/current/bin/node
npm: /home/codespace/nvm/current/bin/npm
npx: /home/codespace/nvm/current/bin/npx
node => v24.11.1
npm => 11.6.2
npx => 11.6.2


In [7]:
import subprocess, pathlib, time, json, os, signal
repo = pathlib.Path('/workspaces/prime-detail')
report_json = repo / 'tools' / 'lighthouse-mobile.json'
report_html = repo / 'tools' / 'lighthouse-mobile.html'

server = subprocess.Popen(
    ['python3', '-m', 'http.server', '4173', '--bind', '127.0.0.1'],
    cwd=str(repo),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

try:
    time.sleep(1.5)
    cmd = [
        'npx', '--yes', 'lighthouse',
        'http://127.0.0.1:4173/index.html',
        '--only-categories=performance,accessibility,best-practices,seo',
        '--emulated-form-factor=mobile',
        '--screenEmulation.mobile',
        '--quiet',
        '--chrome-flags=--headless=new --no-sandbox --disable-dev-shm-usage',
        '--output=json', '--output=html',
        f'--output-path={report_json.with_suffix("")}',
    ]
    run = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True, timeout=420)
    print('exit:', run.returncode)
    if run.stdout.strip():
        print(run.stdout[:4000])
    if run.stderr.strip():
        print(run.stderr[:4000])
finally:
    server.terminate()
    try:
        server.wait(timeout=5)
    except Exception:
        server.kill()

print('json exists:', report_json.exists())
print('html exists:', report_html.exists())
if report_json.exists():
    data = json.loads(report_json.read_text(encoding='utf-8'))
    cats = data.get('categories', {})
    for key in ['performance', 'accessibility', 'best-practices', 'seo']:
        score = cats.get(key, {}).get('score')
        if score is not None:
            print(key, int(score * 100))

exit: 1
Runtime error encountered: The CHROME_PATH environment variable must be set to a Chrome/Chromium executable no older than Chrome stable.
Error
    at new LauncherError (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/utils.js:22:22)
    at new ChromePathNotSetError (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/utils.js:28:9)
    at Module.linux (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/chrome-finder.js:124:15)
    at Launcher.getFirstInstallation (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/chrome-launcher.js:135:43)
    at Launcher.launch (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/chrome-launcher.js:203:43)
    at Module.launch (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/chrome-launcher.js:33:20)
    at getDebuggableChrome (file:///home/cod

In [8]:
import shutil, subprocess
candidates = ['google-chrome', 'google-chrome-stable', 'chromium', 'chromium-browser', 'microsoft-edge', 'brave-browser']
for c in candidates:
    print(c, '=>', shutil.which(c))
print('\nwhich search:')
print(subprocess.run(['bash','-lc','command -v google-chrome google-chrome-stable chromium chromium-browser microsoft-edge brave-browser || true'], capture_output=True, text=True).stdout)

google-chrome => None
google-chrome-stable => None
chromium => None
chromium-browser => None
microsoft-edge => None
brave-browser => None

which search:



In [9]:
import subprocess
cmd = ['bash','-lc','set -e; sudo apt-get update -y && sudo apt-get install -y chromium || apt-get update -y && apt-get install -y chromium']
p = subprocess.run(cmd, capture_output=True, text=True)
print('exit:', p.returncode)
print((p.stdout or '')[-4000:])
print((p.stderr or '')[-4000:])

exit: 100
Get:1 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble InRelease [3600 B]
Get:2 http://archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:3 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]
Get:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease [3961 B]
Get:5 https://dl.yarnpkg.com/debian stable InRelease
Get:6 http://archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:7 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble/main all Packages [643 B]
Get:8 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble/main amd64 Packages [97.6 kB]
Get:9 http://archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:10 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable/main amd64 Packages [4557 B]
Get:11 http://security.ubuntu.com/ubuntu noble-security/restricted amd64 Packages [3262 kB]
Err:5 https://dl.yarnpkg.com/debian stable InRelease
  The following signatures 

In [10]:
import subprocess, pathlib, json
repo = pathlib.Path('/workspaces/prime-detail')
# Install Playwright chromium in user space
install = subprocess.run(['npx','--yes','playwright','install','chromium'], cwd=str(repo), capture_output=True, text=True)
print('install exit:', install.returncode)
print((install.stdout or '')[-3000:])
print((install.stderr or '')[-3000:])

# Resolve executable path using playwright
node_code = "const { chromium } = require('playwright'); const p = chromium.executablePath(); console.log(p);"
path_res = subprocess.run(['node','-e', node_code], cwd=str(repo), capture_output=True, text=True)
print('path exit:', path_res.returncode)
print('chromium path:', (path_res.stdout or '').strip())
print((path_res.stderr or '')[-1000:])

install exit: 0
■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100% of 167.3 MiB
Chrome for Testing 145.0.7632.6 (playwright chromium v1208) downloaded to /home/codespace/.cache/ms-playwright/chromium-1208
|                                                                                |   0% of 2.3 MiB
|■■■■■■■■                                                                        |  10% of 2.3 MiB
|■■■■■■■■■■■■■■■■                                                                |  20% of 2.3 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■                                                        |  30% of 2.3 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                                |  40% of 2.3 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                        |  50% of 2.3 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                |  60% of 2.3 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                        |  70% of 2

In [11]:
import subprocess, pathlib, time, json, os, shutil
repo = pathlib.Path('/workspaces/prime-detail')
report_json = repo / 'tools' / 'lighthouse-mobile.json'
report_html = repo / 'tools' / 'lighthouse-mobile.html'

# Recherche d'un navigateur Chromium/Chrome utilisable sans planter la cellule
candidates = list(pathlib.Path('/home/codespace/.cache/ms-playwright').glob('chromium-*/chrome-linux/chrome'))
playwright_chrome = str(sorted(candidates)[-1]) if candidates else None
system_chrome = shutil.which('google-chrome') or shutil.which('chromium') or shutil.which('chromium-browser')
chrome_path = playwright_chrome or system_chrome

if not chrome_path:
    print('Aucun Chromium/Chrome détecté. Audit Lighthouse ignoré (pas d\'échec bloquant).')
    print('Installez-en un puis relancez cette cellule, ou utilisez tools/run-lighthouse-mobile-once.sh')
else:
    print('CHROME_PATH=', chrome_path)

    server = subprocess.Popen(
        ['python3', '-m', 'http.server', '4173', '--bind', '127.0.0.1'],
        cwd=str(repo),
        stdout=subprocess.DEVNULL,
        stderr=subprocess.STDOUT,
        text=True,
    )

    try:
        time.sleep(1.2)
        cmd = [
            'npx', '--yes', 'lighthouse',
            'http://127.0.0.1:4173/index.html',
            '--only-categories=performance,accessibility,best-practices,seo',
            '--emulated-form-factor=mobile',
            '--screenEmulation.mobile',
            '--quiet',
            '--chrome-flags=--headless=new --no-sandbox --disable-dev-shm-usage',
            '--output=json', '--output=html',
            f'--output-path={report_json.with_suffix("")}',
        ]
        env = os.environ.copy()
        env['CHROME_PATH'] = chrome_path
        run = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True, timeout=420, env=env)
        print('exit:', run.returncode)
        if run.stdout.strip():
            print(run.stdout[:2500])
        if run.stderr.strip():
            print(run.stderr[:2500])
    finally:
        server.terminate()
        try:
            server.wait(timeout=5)
        except Exception:
            server.kill()

    print('json exists:', report_json.exists())
    print('html exists:', report_html.exists())
    if report_json.exists():
        data = json.loads(report_json.read_text(encoding='utf-8'))
        cats = data.get('categories', {})
        audits = data.get('audits', {})
        for key in ['performance', 'accessibility', 'best-practices', 'seo']:
            score = cats.get(key, {}).get('score')
            if score is not None:
                print(key, int(score * 100))
        for k in ['largest-contentful-paint', 'cumulative-layout-shift', 'interactive', 'total-blocking-time']:
            a = audits.get(k, {})
            if 'displayValue' in a:
                print(k, a['displayValue'])

RuntimeError: Chromium Playwright introuvable

In [12]:
import pathlib
root = pathlib.Path('/home/codespace/.cache/ms-playwright')
print('root exists:', root.exists())
for p in sorted(root.glob('*')):
    print(p)
print('candidates recursive:')
for p in sorted(root.rglob('chrome')):
    print(p)

root exists: True
/home/codespace/.cache/ms-playwright/.links
/home/codespace/.cache/ms-playwright/chromium-1208
/home/codespace/.cache/ms-playwright/chromium_headless_shell-1208
/home/codespace/.cache/ms-playwright/ffmpeg-1011
candidates recursive:
/home/codespace/.cache/ms-playwright/chromium-1208/chrome-linux64/chrome


In [13]:
import subprocess, pathlib, time, json, os
repo = pathlib.Path('/workspaces/prime-detail')
report_json = repo / 'tools' / 'lighthouse-mobile.json'
report_html = repo / 'tools' / 'lighthouse-mobile.html'
chrome_path = '/home/codespace/.cache/ms-playwright/chromium-1208/chrome-linux64/chrome'
print('CHROME_PATH=', chrome_path)

server = subprocess.Popen(
    ['python3', '-m', 'http.server', '4173', '--bind', '127.0.0.1'],
    cwd=str(repo),
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT,
    text=True,
)

try:
    time.sleep(1.2)
    cmd = [
        'npx', '--yes', 'lighthouse',
        'http://127.0.0.1:4173/index.html',
        '--only-categories=performance,accessibility,best-practices,seo',
        '--emulated-form-factor=mobile',
        '--screenEmulation.mobile',
        '--quiet',
        '--chrome-flags=--headless=new --no-sandbox --disable-dev-shm-usage',
        '--output=json', '--output=html',
        f'--output-path={report_json.with_suffix("")}',
    ]
    env = os.environ.copy()
    env['CHROME_PATH'] = chrome_path
    run = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True, timeout=420, env=env)
    print('exit:', run.returncode)
    if run.stdout.strip():
        print(run.stdout[:2500])
    if run.stderr.strip():
        print(run.stderr[:2500])
finally:
    server.terminate()
    try:
        server.wait(timeout=5)
    except Exception:
        server.kill()

print('json exists:', report_json.exists())
print('html exists:', report_html.exists())
if report_json.exists():
    data = json.loads(report_json.read_text(encoding='utf-8'))
    cats = data.get('categories', {})
    audits = data.get('audits', {})
    for key in ['performance', 'accessibility', 'best-practices', 'seo']:
        score = cats.get(key, {}).get('score')
        if score is not None:
            print(key, int(score * 100))
    for k in ['largest-contentful-paint', 'cumulative-layout-shift', 'interactive', 'total-blocking-time']:
        a = audits.get(k, {})
        if 'displayValue' in a:
            print(k, a['displayValue'])

CHROME_PATH= /home/codespace/.cache/ms-playwright/chromium-1208/chrome-linux64/chrome
exit: 1
Unable to connect to Chrome

json exists: False
html exists: False


In [14]:
import pathlib
root = pathlib.Path('/home/codespace/.cache/ms-playwright')
for p in sorted(root.rglob('*headless*')):
    if p.is_file():
        print(p)

/home/codespace/.cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/LICENSE.headless_shell
/home/codespace/.cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/chrome-headless-shell
/home/codespace/.cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/headless_command_resources.pak
/home/codespace/.cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/headless_lib_data.pak
/home/codespace/.cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/headless_lib_strings.pak


In [15]:
import subprocess, pathlib, time, json
repo = pathlib.Path('/workspaces/prime-detail')
report_json = repo / 'tools' / 'lighthouse-mobile.json'
report_html = repo / 'tools' / 'lighthouse-mobile.html'
chrome_path = '/home/codespace/.cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/chrome-headless-shell'

server = subprocess.Popen(
    ['python3', '-m', 'http.server', '4173', '--bind', '127.0.0.1'],
    cwd=str(repo),
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT,
    text=True,
)

try:
    time.sleep(1.2)
    cmd = [
        'npx', '--yes', 'lighthouse',
        'http://127.0.0.1:4173/index.html',
        '--chrome-path=' + chrome_path,
        '--only-categories=performance,accessibility,best-practices,seo',
        '--emulated-form-factor=mobile',
        '--screenEmulation.mobile',
        '--quiet',
        '--chrome-flags=--headless --no-sandbox --disable-dev-shm-usage --disable-gpu --remote-debugging-port=9222',
        '--output=json', '--output=html',
        f'--output-path={report_json.with_suffix("")}',
    ]
    run = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True, timeout=420)
    print('exit:', run.returncode)
    if run.stdout.strip():
        print(run.stdout[:2500])
    if run.stderr.strip():
        print(run.stderr[:2500])
finally:
    server.terminate()
    try:
        server.wait(timeout=5)
    except Exception:
        server.kill()

print('json exists:', report_json.exists())
print('html exists:', report_html.exists())
if report_json.exists():
    data = json.loads(report_json.read_text(encoding='utf-8'))
    cats = data.get('categories', {})
    for key in ['performance', 'accessibility', 'best-practices', 'seo']:
        score = cats.get(key, {}).get('score')
        if score is not None:
            print(key, int(score * 100))

exit: 1
Runtime error encountered: The CHROME_PATH environment variable must be set to a Chrome/Chromium executable no older than Chrome stable.
Error
    at new LauncherError (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/utils.js:22:22)
    at new ChromePathNotSetError (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/utils.js:28:9)
    at Module.linux (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/chrome-finder.js:124:15)
    at Launcher.getFirstInstallation (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/chrome-launcher.js:135:43)
    at Launcher.launch (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/chrome-launcher.js:203:43)
    at Module.launch (file:///home/codespace/.npm/_npx/0f94ee7615faf582/node_modules/chrome-launcher/dist/chrome-launcher.js:33:20)
    at getDebuggableChrome (file:///home/cod

In [16]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
p = subprocess.run(['git','-C',str(repo),'checkout','--','tools/recover.ipynb'], capture_output=True, text=True)
print('exit:', p.returncode)
print(p.stdout)
print(p.stderr)

exit: 0




In [27]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
cmd = ['bash', 'tools/run-lighthouse-mobile-once.sh']
res = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
print('exit:', res.returncode)
if res.stdout:
    print(res.stdout[-4000:])
if res.stderr:
    print(res.stderr[-4000:])

exit: 0
Audit Lighthouse mobile sur: http://127.0.0.1:4173/index.html
Sorties:
- /workspaces/prime-detail/tools/lighthouse-mobile.report.html
- /workspaces/prime-detail/tools/lighthouse-mobile.report.json
Terminé ✅
Ouverture du rapport: file:///workspaces/prime-detail/tools/lighthouse-mobile.report.html

r focus is not accidentally trapped in a region
2026-03-02T01:45:44.751Z LH:status Auditing: Interactive controls are keyboard focusable
2026-03-02T01:45:44.752Z LH:status Auditing: Interactive elements indicate their purpose and state
2026-03-02T01:45:44.752Z LH:status Auditing: The page has a logical tab order
2026-03-02T01:45:44.752Z LH:status Auditing: The user's focus is directed to new content added to the page
2026-03-02T01:45:44.752Z LH:status Auditing: Offscreen content is hidden from assistive technology
2026-03-02T01:45:44.752Z LH:status Auditing: HTML5 landmark elements are used to improve navigation
2026-03-02T01:45:44.752Z LH:status Auditing: Visual order on the page foll

In [19]:
import subprocess, pathlib
root = pathlib.Path('/workspaces/prime-detail/.cache/ms-playwright')
print('exists:', root.exists())
res = subprocess.run(['find', str(root), '-maxdepth', '4', '-type', 'f', '-name', 'chrome'], capture_output=True, text=True)
print(res.stdout if res.stdout else '(no chrome file found)')
res2 = subprocess.run(['find', str(root), '-maxdepth', '4', '-type', 'd'], capture_output=True, text=True)
print('\n'.join(res2.stdout.splitlines()[:40]))

exists: True
/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/chrome

/workspaces/prime-detail/.cache/ms-playwright
/workspaces/prime-detail/.cache/ms-playwright/chromium-1208
/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64
/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/resources
/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/resources/inspector_overlay
/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/resources/accessibility
/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm
/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/_platform_specific
/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/MEIPreload
/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/PrivacySandboxAttestationsPreloaded
/workspaces/prime-detail/.cache/ms-playwright/

In [25]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
cmd = ['npx', '--yes', 'playwright@latest', 'install-deps', 'chromium']
res = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
print('exit:', res.returncode)
if res.stdout:
    print(res.stdout[-4000:])
if res.stderr:
    print(res.stderr[-4000:])

exit: 0
ibxdamage1:amd64 (1:1.1.6-1build1) ...
Setting up libxcb-xfixes0:amd64 (1.15-1ubuntu2) ...
Setting up libxpm4:amd64 (1:3.5.17-1build2) ...
Setting up libxi6:amd64 (2:1.8.1-1build1) ...
Setting up fonts-noto-color-emoji (2.047-0ubuntu0.24.04.1) ...
Setting up libglvnd0:amd64 (1.7.0-1build1) ...
Setting up libxcb-glx0:amd64 (1.15-1ubuntu2) ...
Setting up libsensors-config (1:3.6.0-9build1) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-freefont-ttf (20211204+svn4273-2) ...
Setting up xkb-data (2.41-2ubuntu1.1) ...
Setting up libxxf86vm1:amd64 (1:1.1.4-1build4) ...
Setting up libxcb-present0:amd64 (1.15-1ubuntu2) ...
Setting up libasound2-data (1.2.11-1ubuntu0.2) ...
Setting up libfontenc1:amd64 (1:1.1.8-1build1) ...
Setting up libglib2.0-0t64:amd64 (2.80.0-6ubuntu3.8) ...
No schema files found: doing nothing.
Setting up libglib2.0-data (2.80.0-6ubuntu3.8) ...
Setting up libasound2t64:amd64 (1.2.11-1ubuntu0.2) ...
Setting up fonts-tlwg-loma-otf (1:0.7.3-1) ...
Set

In [22]:
import subprocess
cmd = ['bash', '-lc', 'ls -1 /etc/apt/sources.list.d && echo --- && grep -R "yarnpkg" -n /etc/apt/sources.list /etc/apt/sources.list.d 2>/dev/null || true']
res = subprocess.run(cmd, capture_output=True, text=True)
print('exit:', res.returncode)
print(res.stdout)
if res.stderr:
    print(res.stderr)

exit: 0
conda.list
microsoft.list
ubuntu.sources
yarn.list
---
/etc/apt/sources.list.d/yarn.list:1:deb [arch=amd64 signed-by=/usr/share/keyrings/yarn-archive-keyring.gpg] https://dl.yarnpkg.com/debian/ stable main



In [23]:
import subprocess
cmd = [
    'bash', '-lc',
    "set -euo pipefail; curl -fsSL https://dl.yarnpkg.com/debian/pubkey.gpg | sudo gpg --dearmor -o /usr/share/keyrings/yarn-archive-keyring.gpg; sudo apt-get update -y"
 ]
res = subprocess.run(cmd, capture_output=True, text=True)
print('exit:', res.returncode)
if res.stdout:
    print(res.stdout[-3000:])
if res.stderr:
    print(res.stderr[-3000:])

exit: 2
gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failure writing output to destination



In [24]:
import subprocess
cmd = [
    'bash', '-lc',
    "set -euo pipefail; curl -fsSL https://dl.yarnpkg.com/debian/pubkey.gpg | gpg --dearmor --batch --yes | sudo tee /usr/share/keyrings/yarn-archive-keyring.gpg >/dev/null; sudo apt-get update -y"
 ]
res = subprocess.run(cmd, capture_output=True, text=True)
print('exit:', res.returncode)
if res.stdout:
    print(res.stdout[-3000:])
if res.stderr:
    print(res.stderr[-3000:])

exit: 0
Hit:1 http://archive.ubuntu.com/ubuntu noble InRelease
Hit:2 http://archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu noble-backports InRelease
Get:4 https://dl.yarnpkg.com/debian stable InRelease
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:6 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble InRelease
Hit:7 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Get:8 https://dl.yarnpkg.com/debian stable/main all Packages [11.7 kB]
Get:9 https://dl.yarnpkg.com/debian stable/main amd64 Packages [11.7 kB]
Fetched 39.9 kB in 1s (56.1 kB/s)
Reading package lists...



In [28]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
res = subprocess.run(['bash','-lc','cd /workspaces/prime-detail && ls -l tools/run-lighthouse-mobile.sh tools/run-lighthouse-mobile-once.sh'], capture_output=True, text=True)
print('exit:', res.returncode)
print(res.stdout)
if res.stderr:
    print(res.stderr)

exit: 0
-rw-rw-rw- 1 codespace codespace 2496 Mar  2 01:46 tools/run-lighthouse-mobile-once.sh
-rw-rw-rw- 1 codespace codespace 2079 Mar  2 01:46 tools/run-lighthouse-mobile.sh



In [29]:
import subprocess
res = subprocess.run(['bash','-lc','cd /workspaces/prime-detail && chmod +x tools/run-lighthouse-mobile.sh tools/run-lighthouse-mobile-once.sh && ls -l tools/run-lighthouse-mobile.sh tools/run-lighthouse-mobile-once.sh'], capture_output=True, text=True)
print('exit:', res.returncode)
print(res.stdout)
if res.stderr:
    print(res.stderr)

exit: 0
-rwxrwxrwx 1 codespace codespace 2496 Mar  2 01:46 tools/run-lighthouse-mobile-once.sh
-rwxrwxrwx 1 codespace codespace 2079 Mar  2 01:46 tools/run-lighthouse-mobile.sh



In [30]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
res = subprocess.run(['./tools/run-lighthouse-mobile-once.sh'], cwd=str(repo), capture_output=True, text=True)
print('exit:', res.returncode)
print((res.stdout or '')[-1200:])
if res.stderr:
    print((res.stderr or '')[-1200:])

exit: 0
Audit Lighthouse mobile sur: http://127.0.0.1:4173/index.html
Sorties:
- /workspaces/prime-detail/tools/lighthouse-mobile.report.html
- /workspaces/prime-detail/tools/lighthouse-mobile.report.json
Terminé ✅
Ouverture du rapport: file:///workspaces/prime-detail/tools/lighthouse-mobile.report.html

isplay
2026-03-02T01:54:49.895Z LH:status Auditing: Forced reflow
2026-03-02T01:54:49.901Z LH:status Auditing: Improve image delivery
2026-03-02T01:54:49.908Z LH:status Auditing: INP breakdown
2026-03-02T01:54:49.911Z LH:status Auditing: LCP breakdown
2026-03-02T01:54:49.912Z LH:status Auditing: LCP request discovery
2026-03-02T01:54:49.916Z LH:status Auditing: Legacy JavaScript
2026-03-02T01:54:49.920Z LH:status Auditing: Modern HTTP
2026-03-02T01:54:49.923Z LH:status Auditing: Network dependency tree
2026-03-02T01:54:49.927Z LH:status Auditing: Render blocking requests
2026-03-02T01:54:49.929Z LH:status Auditing: 3rd parties
2026-03-02T01:54:49.939Z LH:status Auditing: Optimize viewp

In [32]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')

def run(cmd):
    r = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
    print('$', ' '.join(cmd))
    print(r.stdout.strip())
    if r.stderr.strip():
        print(r.stderr.strip())
    print('---')
    return r

run(['git', 'remote', '-v'])
run(['git', 'status', '--short'])
run(['git', 'config', '--get', 'http.version'])
run(['git', 'config', '--get', 'http.postBuffer'])
run(['git', 'config', '--get', 'http.lowSpeedLimit'])
run(['git', 'config', '--get', 'http.lowSpeedTime'])

ls = subprocess.run(['git', 'ls-files'], cwd=str(repo), capture_output=True, text=True)
sizes = []
for p in ls.stdout.splitlines():
    fp = repo / p
    if fp.is_file():
        sizes.append((fp.stat().st_size, p))
for size, p in sorted(sizes, reverse=True)[:20]:
    print(f'{size:>12}  {p}')

$ git remote -v
origin	https://github.com/Prime-Detail/prime-detail (fetch)
origin	https://github.com/Prime-Detail/prime-detail (push)
---
$ git status --short

---
$ git config --get http.version

---
$ git config --get http.postBuffer

---
$ git config --get http.lowSpeedLimit

---
$ git config --get http.lowSpeedTime

---
   269778136  .cache/ms-playwright/chromium-1208/chrome-linux64/chrome
   183557344  .cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/chrome-headless-shell
    18252696  .cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/_platform_specific/linux_x64/libwidevinecdm.so
    16025326  assets/images/gallery/video.mp4
    12880876  .cache/ms-playwright/chromium-1208/chrome-linux64/resources.pak
    10822192  .cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/icudtl.dat
    10822192  .cache/ms-playwright/chromium-1208/chrome-linux64/icudtl.dat
     6512048  .cache/ms-playwright/chromium_headless_shell-

In [33]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')

def run(cmd):
    r = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
    print('$', ' '.join(cmd))
    if r.stdout.strip():
        print(r.stdout.strip())
    if r.stderr.strip():
        print(r.stderr.strip())
    print('exit:', r.returncode)
    print('---')
    return r

run(['git', 'log', '--oneline', '-n', '5'])
run(['git', 'show', '--name-only', '--pretty=format:', 'HEAD'])

$ git log --oneline -n 5
b09f1bc Update script permissions for run-lighthouse-mobile-once.sh and run-lighthouse-mobile.sh to make them executable
2fe0c67 feat: enhance Lighthouse mobile scripts with Chrome detection and report output updates
d3a59c8 Ajout du fichier `env.d.ts` pour la déclaration des types Astro et mise à jour de `tsconfig.json` pour inclure ce fichier dans la compilation TypeScript.
a504775 Migration de la page `mentions-legales` vers un layout Astro natif et ajout de contenu légal complet. Suppression des scripts inline obsolètes et mise à jour des instructions de build pour la gestion des assets.
c30ef04 Ajout de nouveaux scripts pour la gestion des formulaires de contact, des vidéos et du quiz tarifaire.
exit: 0
---
$ git show --name-only --pretty=format: HEAD
README.md
tools/lighthouse-mobile.report.html
tools/lighthouse-mobile.report.json
tools/run-lighthouse-mobile-once.sh
tools/run-lighthouse-mobile.sh
exit: 0
---


CompletedProcess(args=['git', 'show', '--name-only', '--pretty=format:', 'HEAD'], returncode=0, stdout='README.md\ntools/lighthouse-mobile.report.html\ntools/lighthouse-mobile.report.json\ntools/run-lighthouse-mobile-once.sh\ntools/run-lighthouse-mobile.sh\n', stderr='')

In [34]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')

def run(cmd):
    r = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
    print('$', ' '.join(cmd))
    if r.stdout.strip():
        print(r.stdout.strip())
    if r.stderr.strip():
        print(r.stderr.strip())
    print('exit:', r.returncode)
    print('---')
    return r

run(['git', 'rm', '-r', '--cached', '.cache'])
run(['git', 'rm', '--cached', 'tools/lighthouse-mobile.report.html', 'tools/lighthouse-mobile.report.json'])
run(['git', 'add', '.gitignore'])
run(['git', 'status', '--short'])

$ git rm -r --cached .cache
rm '.cache/ms-playwright/.links/6ce9bcf0a539f32f8ff0fbc823ebc2032b91a0ca'
rm '.cache/ms-playwright/chromium-1208/DEPENDENCIES_VALIDATED'
rm '.cache/ms-playwright/chromium-1208/INSTALLATION_COMPLETE'
rm '.cache/ms-playwright/chromium-1208/chrome-linux64/ABOUT'
rm '.cache/ms-playwright/chromium-1208/chrome-linux64/MEIPreload/manifest.json'
rm '.cache/ms-playwright/chromium-1208/chrome-linux64/MEIPreload/preloaded_data.pb'
rm '.cache/ms-playwright/chromium-1208/chrome-linux64/PrivacySandboxAttestationsPreloaded/manifest.json'
rm '.cache/ms-playwright/chromium-1208/chrome-linux64/PrivacySandboxAttestationsPreloaded/privacy-sandbox-attestations.dat'
rm '.cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/LICENSE'
rm '.cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/_platform_specific/linux_x64/libwidevinecdm.so'
rm '.cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/manifest.json'
rm '.cache/ms-playwright/chromium-1208/chrome-lin

CompletedProcess(args=['git', 'status', '--short'], returncode=0, stdout='D  .cache/ms-playwright/.links/6ce9bcf0a539f32f8ff0fbc823ebc2032b91a0ca\nD  .cache/ms-playwright/chromium-1208/DEPENDENCIES_VALIDATED\nD  .cache/ms-playwright/chromium-1208/INSTALLATION_COMPLETE\nD  .cache/ms-playwright/chromium-1208/chrome-linux64/ABOUT\nD  .cache/ms-playwright/chromium-1208/chrome-linux64/MEIPreload/manifest.json\nD  .cache/ms-playwright/chromium-1208/chrome-linux64/MEIPreload/preloaded_data.pb\nD  .cache/ms-playwright/chromium-1208/chrome-linux64/PrivacySandboxAttestationsPreloaded/manifest.json\nD  .cache/ms-playwright/chromium-1208/chrome-linux64/PrivacySandboxAttestationsPreloaded/privacy-sandbox-attestations.dat\nD  .cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/LICENSE\nD  .cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/_platform_specific/linux_x64/libwidevinecdm.so\nD  .cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/manifest.json\nD  .cache/ms-p

In [35]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
for cmd in [
    ['git','status','--short','--branch'],
    ['git','ls-files','.cache'],
    ['git','ls-files','tools/lighthouse-mobile.report.html','tools/lighthouse-mobile.report.json'],
    ['bash','-lc','test -f /workspaces/prime-detail/.gitignore && echo .gitignore_present || echo .gitignore_missing']
 ]:
    r = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
    print('$', ' '.join(cmd))
    print(r.stdout.strip() or '(empty)')
    if r.stderr.strip():
        print(r.stderr.strip())
    print('---')

$ git status --short --branch
## main...origin/main [ahead 2]
D  .cache/ms-playwright/.links/6ce9bcf0a539f32f8ff0fbc823ebc2032b91a0ca
D  .cache/ms-playwright/chromium-1208/DEPENDENCIES_VALIDATED
D  .cache/ms-playwright/chromium-1208/INSTALLATION_COMPLETE
D  .cache/ms-playwright/chromium-1208/chrome-linux64/ABOUT
D  .cache/ms-playwright/chromium-1208/chrome-linux64/MEIPreload/manifest.json
D  .cache/ms-playwright/chromium-1208/chrome-linux64/MEIPreload/preloaded_data.pb
D  .cache/ms-playwright/chromium-1208/chrome-linux64/PrivacySandboxAttestationsPreloaded/manifest.json
D  .cache/ms-playwright/chromium-1208/chrome-linux64/PrivacySandboxAttestationsPreloaded/privacy-sandbox-attestations.dat
D  .cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/LICENSE
D  .cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/_platform_specific/linux_x64/libwidevinecdm.so
D  .cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/manifest.json
D  .cache/ms-playwright/chromium-1208

In [36]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
cmd = [
  'bash','-lc',
  'cd /workspaces/prime-detail && git status --short --branch && echo --- && git ls-files .cache | head -n 20 && echo --- && git ls-files tools/lighthouse-mobile.report.html tools/lighthouse-mobile.report.json && echo --- && ls -l .gitignore'
 ]
r = subprocess.run(cmd, capture_output=True, text=True)
print('exit:', r.returncode)
print(r.stdout)
if r.stderr:
    print(r.stderr)

exit: 0
## main...origin/main [ahead 2]
D  .cache/ms-playwright/.links/6ce9bcf0a539f32f8ff0fbc823ebc2032b91a0ca
D  .cache/ms-playwright/chromium-1208/DEPENDENCIES_VALIDATED
D  .cache/ms-playwright/chromium-1208/INSTALLATION_COMPLETE
D  .cache/ms-playwright/chromium-1208/chrome-linux64/ABOUT
D  .cache/ms-playwright/chromium-1208/chrome-linux64/MEIPreload/manifest.json
D  .cache/ms-playwright/chromium-1208/chrome-linux64/MEIPreload/preloaded_data.pb
D  .cache/ms-playwright/chromium-1208/chrome-linux64/PrivacySandboxAttestationsPreloaded/manifest.json
D  .cache/ms-playwright/chromium-1208/chrome-linux64/PrivacySandboxAttestationsPreloaded/privacy-sandbox-attestations.dat
D  .cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/LICENSE
D  .cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/_platform_specific/linux_x64/libwidevinecdm.so
D  .cache/ms-playwright/chromium-1208/chrome-linux64/WidevineCdm/manifest.json
D  .cache/ms-playwright/chromium-1208/chrome-linux64/chrome

In [37]:
import subprocess
cmd = [
  'bash','-lc',
  'cd /workspaces/prime-detail && echo STATUS_COUNT=$(git status --short | wc -l) && echo CACHE_TRACKED_COUNT=$(git ls-files .cache | wc -l) && echo REPORT_TRACKED_COUNT=$(git ls-files tools/lighthouse-mobile.report.html tools/lighthouse-mobile.report.json | wc -l) && test -f .gitignore && echo GITIGNORE=1 || echo GITIGNORE=0'
 ]
r = subprocess.run(cmd, capture_output=True, text=True)
print('exit:', r.returncode)
print(r.stdout)
if r.stderr:
    print(r.stderr)

exit: 0
STATUS_COUNT=604
CACHE_TRACKED_COUNT=0
REPORT_TRACKED_COUNT=0
GITIGNORE=1



In [38]:
import subprocess
cmd = ['bash','-lc','cd /workspaces/prime-detail && git fetch origin --quiet || true; echo AHEAD_BEHIND=$(git rev-list --left-right --count origin/main...HEAD 2>/dev/null || echo unknown) && echo FIRST_CACHE_COMMIT=$(git log --all --pretty=format:%h -- .cache | tail -n 1) && echo LAST_CACHE_COMMIT=$(git log --all --pretty=format:%h -n 1 -- .cache)']
r = subprocess.run(cmd, capture_output=True, text=True)
print('exit:', r.returncode)
print(r.stdout)
if r.stderr:
    print(r.stderr)

exit: 0
AHEAD_BEHIND=0 2
FIRST_CACHE_COMMIT=2fe0c67
LAST_CACHE_COMMIT=2fe0c67



In [39]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')

commands = [
  ['git','branch','backup-rpc-408-fix'],
  ['git','reset','--hard','origin/main'],
  ['git','checkout','2fe0c67','--','tools/run-lighthouse-mobile-once.sh','tools/run-lighthouse-mobile.sh','README.md'],
  ['bash','-lc','cd /workspaces/prime-detail && chmod +x tools/run-lighthouse-mobile-once.sh tools/run-lighthouse-mobile.sh'],
  ['bash','-lc','cd /workspaces/prime-detail && printf ".cache/\ntools/lighthouse-mobile.report.html\ntools/lighthouse-mobile.report.json\n" > .gitignore'],
  ['git','add','.gitignore','README.md','tools/run-lighthouse-mobile-once.sh','tools/run-lighthouse-mobile.sh'],
  ['git','commit','-m','fix: remove cache artifacts and stabilize lighthouse scripts for push'],
  ['git','status','--short','--branch'],
  ['bash','-lc','cd /workspaces/prime-detail && git rev-list --left-right --count origin/main...HEAD']
 ]

for cmd in commands:
    r = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
    print('$', ' '.join(cmd))
    if r.stdout.strip():
        print(r.stdout.strip())
    if r.stderr.strip():
        print(r.stderr.strip())
    print('exit:', r.returncode)
    print('---')

$ git branch backup-rpc-408-fix
exit: 0
---
$ git reset --hard origin/main
HEAD is now at d3a59c8 Ajout du fichier `env.d.ts` pour la déclaration des types Astro et mise à jour de `tsconfig.json` pour inclure ce fichier dans la compilation TypeScript.
exit: 0
---
$ git checkout 2fe0c67 -- tools/run-lighthouse-mobile-once.sh tools/run-lighthouse-mobile.sh README.md
exit: 0
---
$ bash -lc cd /workspaces/prime-detail && chmod +x tools/run-lighthouse-mobile-once.sh tools/run-lighthouse-mobile.sh
exit: 0
---
$ bash -lc cd /workspaces/prime-detail && printf ".cache/
tools/lighthouse-mobile.report.html
tools/lighthouse-mobile.report.json
" > .gitignore
exit: 0
---
$ git add .gitignore README.md tools/run-lighthouse-mobile-once.sh tools/run-lighthouse-mobile.sh
exit: 0
---
$ git commit -m fix: remove cache artifacts and stabilize lighthouse scripts for push
[main 44683d8] fix: remove cache artifacts and stabilize lighthouse scripts for push
 3 files changed, 89 insertions(+), 6 deletions(-)
 c

In [40]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
r = subprocess.run(['git','push','origin','main'], cwd=str(repo), capture_output=True, text=True)
print('exit:', r.returncode)
if r.stdout.strip():
    print(r.stdout.strip())
if r.stderr.strip():
    print(r.stderr.strip())

exit: 0
To https://github.com/Prime-Detail/prime-detail
   d3a59c8..44683d8  main -> main


In [41]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
res = subprocess.run(['bash','tools/pre-push-check.sh'], cwd=str(repo), capture_output=True, text=True)
print('exit:', res.returncode)
print((res.stdout or '')[-5000:])
if res.stderr:
    print((res.stderr or '')[-2000:])

exit: 0
Vérification pre-push (seuil: 50 MB)

1) Fichiers suivis mais ignorés (.gitignore)
OK

2) Fichiers STAGÉS > 50 MB
OK

3) Plus gros fichiers suivis (top 10)
   15.28 MB  assets/images/gallery/video.mp4
    2.89 MB  assets/images/gallery/polissage.png
    2.79 MB  assets/images/gallery/avant.jpeg
    2.52 MB  assets/images/gallery/banquette1.jpeg
    2.48 MB  assets/images/gallery/siege2.jpeg
    2.36 MB  assets/images/gallery/siege1.jpeg
    2.18 MB  assets/images/gallery/apres.jpeg
    2.14 MB  assets/images/gallery/detail.jpeg
    1.58 MB  assets/images/gallery/banquette2.jpeg
    0.89 MB  assets/images/logo/file:/Users/logo.png

Résultat: OK ✅



In [42]:
import os, stat, pathlib, subprocess
repo = pathlib.Path('/workspaces/prime-detail')
hook = repo / '.git' / 'hooks' / 'pre-push'
print('exists:', hook.exists())
mode = hook.stat().st_mode
print('executable:', bool(mode & stat.S_IXUSR))
print('mode_octal:', oct(mode & 0o777))
res = subprocess.run(['bash', 'tools/pre-push-check.sh'], cwd=str(repo), capture_output=True, text=True)
print('pre-push-check exit:', res.returncode)
print((res.stdout or '')[-1200:])
if res.stderr:
    print((res.stderr or '')[-400:])

exists: True
executable: True
mode_octal: 0o755
pre-push-check exit: 0
Vérification pre-push (seuil: 50 MB)

1) Fichiers suivis mais ignorés (.gitignore)
OK

2) Fichiers STAGÉS > 50 MB
OK

3) Plus gros fichiers suivis (top 10)
   15.28 MB  assets/images/gallery/video.mp4
    2.89 MB  assets/images/gallery/polissage.png
    2.79 MB  assets/images/gallery/avant.jpeg
    2.52 MB  assets/images/gallery/banquette1.jpeg
    2.48 MB  assets/images/gallery/siege2.jpeg
    2.36 MB  assets/images/gallery/siege1.jpeg
    2.18 MB  assets/images/gallery/apres.jpeg
    2.14 MB  assets/images/gallery/detail.jpeg
    1.58 MB  assets/images/gallery/banquette2.jpeg
    0.89 MB  assets/images/logo/file:/Users/logo.png

Résultat: OK ✅



In [43]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail/astro')

for cmd in ([['npm','run','sync:assets'], ['npm','run','build']]):
    res = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
    print('$', ' '.join(cmd), '=>', res.returncode)
    if res.stdout:
        print(res.stdout[-2000:])
    if res.stderr:
        print(res.stderr[-1200:])

$ npm run sync:assets => 0

> prime-detail-astro@0.1.0 sync:assets
> node scripts/sync-assets.mjs

[sync:assets] Copie terminée: /workspaces/prime-detail/assets -> /workspaces/prime-detail/astro/public/assets

$ npm run build => 127

> prime-detail-astro@0.1.0 prebuild
> npm run sync:assets


> prime-detail-astro@0.1.0 sync:assets
> node scripts/sync-assets.mjs

[sync:assets] Copie terminée: /workspaces/prime-detail/assets -> /workspaces/prime-detail/astro/public/assets

> prime-detail-astro@0.1.0 build
> astro build


sh: 1: astro: not found



In [44]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail/astro')

for cmd in ([['npm','install'], ['npm','run','build']]):
    res = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
    print('$', ' '.join(cmd), '=>', res.returncode)
    if res.stdout:
        print(res.stdout[-2500:])
    if res.stderr:
        print(res.stderr[-1200:])

$ npm install => 0

added 277 packages, and audited 278 packages in 57s

149 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities

$ npm run build => 0

> prime-detail-astro@0.1.0 prebuild
> npm run sync:assets


> prime-detail-astro@0.1.0 sync:assets
> node scripts/sync-assets.mjs

[sync:assets] Copie terminée: /workspaces/prime-detail/assets -> /workspaces/prime-detail/astro/public/assets

> prime-detail-astro@0.1.0 build
> astro build

▶ Astro collects anonymous usage data.
  This information helps us improve Astro.
  Run "astro telemetry disable" to opt-out.
  https://astro.build/telemetry

02:10:57 [content] Syncing content
02:10:57 [content] Synced content
02:10:57 [types] Generated 629ms
02:10:57 [build] output: "static"
02:10:57 [build] mode: "static"
02:10:57 [build] directory: /workspaces/prime-detail/astro/dist/
02:10:57 [build] Collecting build info...
02:10:57 [build] ✓ Completed in 718ms.
02:10:57 [build] Building static entrypoints...
02

In [45]:
import subprocess, pathlib, time, json, os
repo = pathlib.Path('/workspaces/prime-detail')
astro_dist = repo / 'astro' / 'dist'
out_base = repo / 'tools' / 'lighthouse-astro-mobile'

# Trouver chrome/chromium
chrome = None
for cand in [
    '/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/chrome',
    '/usr/bin/google-chrome', '/usr/bin/chromium', '/usr/bin/chromium-browser'
 ]:
    if pathlib.Path(cand).exists():
        chrome = cand
        break
if not chrome:
    print('Aucun Chrome/Chromium trouvé')
else:
    print('CHROME_PATH=', chrome)
    server = subprocess.Popen(
        ['python3','-m','http.server','4174','--bind','127.0.0.1'],
        cwd=str(astro_dist), stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, text=True
    )
    try:
        time.sleep(1.2)
        cmd = [
            'npx','--yes','lighthouse','http://127.0.0.1:4174/index.html',
            '--only-categories=performance,accessibility,best-practices,seo',
            '--emulated-form-factor=mobile','--screenEmulation.mobile',
            '--chrome-flags=--headless=new --no-sandbox --disable-dev-shm-usage',
            '--output=json','--output=html',f'--output-path={out_base}'
        ]
        env = os.environ.copy()
        env['CHROME_PATH'] = chrome
        run = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True, timeout=420, env=env)
        print('exit:', run.returncode)
        if run.stderr.strip():
            print(run.stderr[-1200:])
    finally:
        server.terminate()
        try:
            server.wait(timeout=5)
        except Exception:
            server.kill()

    report_json = repo / 'tools' / 'lighthouse-astro-mobile.report.json'
    if report_json.exists():
        data = json.loads(report_json.read_text(encoding='utf-8'))
        cats = data.get('categories', {})
        print('scores astro:')
        for key in ['performance','accessibility','best-practices','seo']:
            score = cats.get(key, {}).get('score')
            if score is not None:
                print(f'- {key}: {int(score*100)}')

CHROME_PATH= /workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/chrome
exit: 1
ED
2026-03-02T02:11:50.255Z LH:status Auditing: Network dependency tree
2026-03-02T02:11:50.256Z LH:network-dependency-tree-insight:warn Caught exception: TARGET_CRASHED
2026-03-02T02:11:50.258Z LH:status Auditing: Render blocking requests
2026-03-02T02:11:50.258Z LH:render-blocking-insight:warn Caught exception: TARGET_CRASHED
2026-03-02T02:11:50.260Z LH:status Auditing: 3rd parties
2026-03-02T02:11:50.263Z LH:third-parties-insight:warn Caught exception: TARGET_CRASHED
2026-03-02T02:11:50.270Z LH:status Auditing: Optimize viewport for mobile
2026-03-02T02:11:50.270Z LH:viewport-insight:warn Caught exception: TARGET_CRASHED
2026-03-02T02:11:50.271Z LH:status Generating results...
2026-03-02T02:11:50.380Z LH:Printer json output written to /workspaces/prime-detail/tools/lighthouse-astro-mobile.report.json
2026-03-02T02:11:50.384Z LH:Printer html output written to /workspaces/prime-detai

In [50]:
import subprocess, pathlib, time, json, os
repo = pathlib.Path('/workspaces/prime-detail')
astro_dist = repo / 'astro' / 'dist'
out_base = repo / 'tools' / 'lighthouse-astro-mobile'
chrome = '/workspaces/prime-detail/.cache/ms-playwright/chromium-1208/chrome-linux64/chrome'

server = subprocess.Popen(
    ['python3','-m','http.server','4174','--bind','127.0.0.1'],
    cwd=str(astro_dist), stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, text=True
 )
try:
    time.sleep(1.2)
    cmd = [
        'npx','--yes','lighthouse','http://127.0.0.1:4174/index.html',
        '--only-categories=performance,accessibility,best-practices,seo',
        '--emulated-form-factor=mobile','--screenEmulation.mobile','--quiet',
        '--chrome-flags=--headless=new --no-sandbox --disable-dev-shm-usage --disable-gpu --disable-software-rasterizer --disable-extensions',
        '--output=json','--output=html',f'--output-path={out_base}'
    ]
    env = os.environ.copy()
    env['CHROME_PATH'] = chrome
    run = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True, timeout=420, env=env)
    print('exit:', run.returncode)
    if run.stdout.strip():
        print(run.stdout[-1200:])
    if run.stderr.strip():
        print(run.stderr[-1200:])
finally:
    server.terminate()
    try:
        server.wait(timeout=5)
    except Exception:
        server.kill()

report_json = repo / 'tools' / 'lighthouse-astro-mobile.report.json'
if report_json.exists():
    data = json.loads(report_json.read_text(encoding='utf-8'))
    cats = data.get('categories', {})
    print('scores astro:')
    for key in ['performance','accessibility','best-practices','seo']:
        score = cats.get(key, {}).get('score')
        print(f'- {key}: {None if score is None else int(score*100)}')

exit: 0
scores astro:
- performance: 68
- accessibility: 96
- best-practices: 100
- seo: 100


In [47]:
import json, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
base_json = repo / 'tools' / 'lighthouse-mobile.report.json'
astro_json = repo / 'tools' / 'lighthouse-astro-mobile.report.json'

def read_scores(path):
    if not path.exists():
        return None
    data = json.loads(path.read_text(encoding='utf-8'))
    cats = data.get('categories', {})
    return {k: int((cats.get(k, {}).get('score') or 0) * 100) for k in ['performance','accessibility','best-practices','seo']}

base = read_scores(base_json)
astro = read_scores(astro_json)
print('static:', base)
print('astro :', astro)
if base and astro:
    print('delta :', {k: astro[k]-base[k] for k in astro})

static: {'performance': 70, 'accessibility': 96, 'best-practices': 100, 'seo': 100}
astro : {'performance': 100, 'accessibility': 59, 'best-practices': 92, 'seo': 80}
delta : {'performance': 30, 'accessibility': -37, 'best-practices': -8, 'seo': -20}


In [48]:
import json, pathlib
report = pathlib.Path('/workspaces/prime-detail/tools/lighthouse-astro-mobile.report.json')
data = json.loads(report.read_text(encoding='utf-8'))
cats = data.get('categories', {})
audits = data.get('audits', {})

for cat in ['accessibility','seo','best-practices']:
    print(f'\n[{cat}]')
    refs = cats.get(cat, {}).get('auditRefs', [])
    lows = []
    for ref in refs:
        aid = ref.get('id')
        score = audits.get(aid, {}).get('score')
        if score is not None and score < 1:
            lows.append((score, aid, audits.get(aid, {}).get('title','')))
    for score, aid, title in sorted(lows)[:20]:
        print(f'- {aid}: {int(score*100)} | {title}')


[accessibility]
- document-title: 0 | Document doesn't have a `<title>` element
- html-has-lang: 0 | `<html>` element does not have a `[lang]` attribute

[seo]
- document-title: 0 | Document doesn't have a `<title>` element
- meta-description: 0 | Document does not have a meta description

[best-practices]
- charset: 0 | Charset declaration is missing or occurs too late in the HTML
- errors-in-console: 0 | Browser errors were logged to the console


In [49]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail/astro')
res = subprocess.run(['npm','run','build'], cwd=str(repo), capture_output=True, text=True)
print('build exit:', res.returncode)
print((res.stdout or '')[-1600:])
if res.stderr:
    print((res.stderr or '')[-800:])

build exit: 0

> prime-detail-astro@0.1.0 prebuild
> npm run sync:assets


> prime-detail-astro@0.1.0 sync:assets
> node scripts/sync-assets.mjs

[sync:assets] Copie terminée: /workspaces/prime-detail/assets -> /workspaces/prime-detail/astro/public/assets

> prime-detail-astro@0.1.0 build
> astro build

02:14:24 [content] Syncing content
02:14:24 [content] Synced content
02:14:24 [types] Generated 226ms
02:14:24 [build] output: "static"
02:14:24 [build] mode: "static"
02:14:24 [build] directory: /workspaces/prime-detail/astro/dist/
02:14:24 [build] Collecting build info...
02:14:24 [build] ✓ Completed in 247ms.
02:14:24 [build] Building static entrypoints...
02:14:28 [vite] ✓ built in 3.99s
02:14:28 [build] ✓ Completed in 4.06s.

 generating static routes 
02:14:28 ▶ src/pages/mentions-legales.astro
02:14:28   └─ /mentions-legales/index.html (+25ms) 
02:14:28 ▶ src/pages/index.astro
02:14:28   └─ /index.html (+9ms) 
02:14:28 ✓ Completed in 49ms.

02:14:28 [build] 2 page(s) built in 4.3

In [51]:
import json, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
base = json.loads((repo/'tools/lighthouse-mobile.report.json').read_text(encoding='utf-8'))['categories']
astro = json.loads((repo/'tools/lighthouse-astro-mobile.report.json').read_text(encoding='utf-8'))['categories']
keys = ['performance','accessibility','best-practices','seo']
print('Comparatif final (static -> astro):')
for k in keys:
    b = int((base[k]['score'] or 0)*100)
    a = int((astro[k]['score'] or 0)*100)
    print(f'- {k}: {b} -> {a} (delta {a-b:+d})')

Comparatif final (static -> astro):
- performance: 70 -> 68 (delta -2)
- accessibility: 96 -> 96 (delta +0)
- best-practices: 100 -> 100 (delta +0)
- seo: 100 -> 100 (delta +0)


In [ ]:
import subprocess, pathlib
repo = pathlib.Path('/workspaces/prime-detail')

def sh(cmd):
    r = subprocess.run(cmd, cwd=str(repo), capture_output=True, text=True)
    print('$', ' '.join(cmd))
    if r.stdout.strip():
        print(r.stdout.strip())
    if r.stderr.strip():
        print(r.stderr.strip())
    print('exit:', r.returncode)
    print('---')
    return r

sh(['git','status','--short','--branch'])
sh(['bash','-lc','cd /workspaces/prime-detail && git diff --cached --name-only | sed -n "1,80p"'])

In [ ]:
import subprocess
r = subprocess.run(['bash','-lc','cd /workspaces/prime-detail && git rev-parse --abbrev-ref HEAD && git rev-list --left-right --count origin/main...HEAD'], capture_output=True, text=True)
print('exit:', r.returncode)
print(r.stdout)
if r.stderr:
    print(r.stderr)